In [218]:
file_name = "utah_3_rightish_test25"
total_num_parties=3
centroid_translate = 25

### Geometry functions

In [219]:
# get a point on a side from distance along that side
def left_y(x):
    if x > 500 or x < 0:
        return None
    return x * np.sqrt(3)

def get_left_point(d):
    return (d / 2, d / 2 * np.sqrt(3))
    
def get_right_point(d):
    l_point = get_left_point(d)
    return (1000 - l_point[0], l_point[1])

def get_bottom_point(d):
    return (d, 0)

get_point = {0: get_bottom_point, 1: get_left_point, 2: get_right_point}

# functions to get points on sides given x
def y_bottom(x):
    return 0
def y_left(x):
    return x * np.sqrt(3)
def y_right(x):
    return np.sqrt(3) * (1000 - x)

# check bounds
def in_bounds(point):
    x, y = point[0], point[1]
    condition1 = y >= y_bottom(x)
    condition2 = y <= y_left(x)
    condition3 = y <= y_right(x)
    return condition1 and condition2 and condition3

# functions to move away from a side of the triangle
def right_side_deltas(d):
    return (-d/2 * np.sqrt(3), -d/2)
def left_side_deltas(d):
    return (d/2 * np.sqrt(3), -d/2)
def bottom_side_deltas(d):
    return (0, d)

get_deltas = {0: bottom_side_deltas, 1: left_side_deltas, 2: right_side_deltas}

### General map making function to build file from data

In [220]:
import numpy as np

population = 333333
pop_1_3    = population/3

def make_coastal(name, locations, total_num_parties, num_coast_parties):
    f = open(name, "w")
    coastal_pop = int(np.floor((total_num_parties - 1) / total_num_parties * population))
    if num_coast_parties==1 and total_num_parties==2:
        prefs_right = [1.0,0.1]
        prefs_left = prefs_right
    if num_coast_parties==2 and total_num_parties==3:
        prefs_left = [1.0, 0.1, 0.1]
        prefs_right = [0.1,1.0,0.1]
    
    line = str(population) + " " + str(total_num_parties) + "\n"
    f.write(line)
    for i in range(coastal_pop):
        location = locations[i]
        loc_x = location[0]
        loc_y = location[1]
        prefs = prefs_right
        if loc_x < 500:
            prefs = prefs_left
        f.write(str(loc_x) + " ")
        f.write(str(loc_y) + " ")
        f.write(str(prefs[0]) + " " + str(prefs[1]) + "\n")

    f.close()

In [221]:
def make_central(name, total_num_parties):
    mean = np.array([centroid.x, centroid.y])
    var = 200
    cov = np.array([[var, 0],
                    [0, var]])

    f = open(name, "a+")
    central_pop = int(np.ceil(population / total_num_parties))
    prefs = [0.1] * (total_num_parties - 1)
    prefs.append(1.0)
    for i in range(central_pop):
        location = np.random.multivariate_normal(mean, cov, size=1)
        loc_x = location[0][0]
        loc_y = location[0][1]
        f.write(str(loc_x) + " ")
        f.write(str(loc_y) + " ")
        pref_string = ""
        for i in range(total_num_parties):
            pref_string += str(prefs[i]) + " "
        f.write(pref_string + "\n")
    f.close()

In [222]:
prefs = [0.1] * (3 - 1)
prefs.append(1.0)
pref_string = ""
for i in range(3):
    pref_string += str(prefs[i]) + " "
pref_string

'0.1 0.1 1.0 '

### Get voter points and voter preferences

In [223]:
make_coastal(file_name, locs, total_num_parties=total_num_parties, num_coast_parties=total_num_parties-1)

In [224]:
def generate_fuzzy_coastal_point():
    while(True):
        side = np.random.randint(3)
        d = np.random.normal(500, 300)
        border_point = get_point[side](d)
        dis_in = np.random.exponential() * 50
        deltas = get_deltas[side](dis_in)
        point = (border_point[0] + deltas[0], border_point[1] + deltas[1])
        if in_bounds(point):
            return point

locs = []
for i in range(222222):
    locs.append(generate_fuzzy_coastal_point())

In [225]:
make_central(file_name, total_num_parties=total_num_parties)

### Print map to file

In [226]:
class Point:
    def __init__(self, x,y):
        self.x = x
        self.y = y
    def __repr__(self):
        return "({}, {})".format(self.x, self.y)

In [227]:
map_points = [Point(0,0), Point(1000,0), Point(500, 500 * np.sqrt(3))]
mean_x = np.mean([map_points[0].x, map_points[1].x, map_points[2].x])
mean_y = np.mean([map_points[0].y, map_points[1].y, map_points[2].y])
centroid = Point(mean_x, mean_y)
rightish_centroid = Point(mean_x + centroid_translate, mean_y)

In [228]:
bottom_linspace = np.linspace(0,1000, 28)
left_linspace = np.linspace(0, 500, 28)
right_linspace = np.linspace(500, 1000, 28)

def get_1_3rd_districts(get_y, x_linspace):
    triangles = []
    for i in range(27):
        triangle = [rightish_centroid]
        x = x_linspace[i]
        y = get_y(x)
        triangle.append(Point(x, y))
        x = x_linspace[i + 1]
        y = get_y(x)
        triangle.append(Point(x, y))
        triangles.append(triangle)
    return triangles

def get_all_3_districts():
    dist1 = get_1_3rd_districts(y_bottom, bottom_linspace)
    dist2 = get_1_3rd_districts(y_left, left_linspace)
    dist3 = get_1_3rd_districts(y_right, right_linspace)
    return dist1 + dist2 + dist3
    

In [229]:
def append_districts(name):
    f = open(name, "a+")
    f.write(str(81) + "\n")
    triangles = get_all_3_districts()
    for triangle in triangles:
        f.write(str(3) + " ")
        for point in triangle:
            f.write(str(point.x) + " " + str(point.y) + " ")
        f.write("\n")
    f.close()
    

In [230]:
append_districts(file_name)